In [1]:
import numpy as np
import torch
import string
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
import math
import torch.utils.data as data
import json
import os
import pandas as pd
import random
import copy
import torch.utils.data.sampler as sampler
import torch.optim.lr_scheduler as lr_scheduler
import pickle


In [6]:
        with open('./data/chat_raw.pkl',"rb") as f1:  
            chat_result=pickle.load(f1)
        
        with open('./data/audio_raw_feature_13.pickle',"rb") as f2:  
            audio=pickle.load(f2)
        with open('./data/video_raw_feature_sum_moving_average.pickle',"rb") as f2:  
            video=pickle.load(f2)

In [28]:
sample = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654','102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630','102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']


In [30]:
for i in sample:
    print('==')
    print(i)
    print(len(chat_result[i]))
    print(len(audio[i]))
    print(len(video[i]))

==
102844412722519367
2073
2069
2072
==
102844212429550795
1476
1472
1475
==
102844401151219358
1938
1934
1938
==
102844401154430631
1656
1652
1656
==
102844412717014335
1909
1906
1909
==
102844401153971877
1780
1777
1779
==
102844224148503678
2078
2074
2077
==
102844412722847048
2159
2158
2158
==
102844401152857762
1447
1443
1446
==
102844412707380528
1617
1611
1617
==
102844212431516886
1822
1818
1821
==
102844283027925085
2271
2267
2270
==
102844412716227901
1749
1745
1749
==
102844412710001974
1822
1818
1822
==
102844294670878922
2200
2196
2199
==
102844294670551241
2346
2343
2346
==
102844283023599703
1477
1473
1477
==
102844412704496937
2633
2630
2633
==
102844235751783874
2421
2416
2420
==
102844401152071328
1865
1860
1864
==
102844412709674293
1864
1860
1863
==
102844401153447587
1770
1767
1770
==
102844224148896895
1808
1805
1808
==
102844235746868664
1987
1982
1986
==
102979081290790284
2336
2331
2335
==
102844283027531868
1788
1784
1788
==
102844212431975640
2365
2361
2364
=

In [39]:
import torch.utils.data as data

class Mul_data(data.Dataset):
    def __init__(self,d_type):
        self.d_type=d_type

        with open('./data/chat_raw.pkl',"rb") as f1:  
            self.chat_result=pickle.load(f1)
        
        with open('./data/audio_raw_feature_13.pickle',"rb") as f2:  
            self.audio=pickle.load(f2)
        with open('./data/video_raw_feature_sum_moving_average.pickle',"rb") as f2:  
            self.video=pickle.load(f2)
        with open('./label/label.pickle',"rb") as f4:  
            self.real_result=pickle.load(f4)
            
        if d_type=='train':
            self.sample = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654']
        if d_type=='val':
            self.sample = ['102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630']
        if d_type=='test':
            self.sample = ['102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']
            
        self.WeightedSampling=[]
        for i in self.sample:
            self.WeightedSampling.extend(copy.copy(self.real_result[str(i)]))
        
        sampling = np.array(self.WeightedSampling)
        neg_idx = np.where(sampling == 0)[0] #general
        pos_idx = np.where(sampling == 1)[0] #highlight
        sampling = sampling.astype(np.float32)
        
        sampling.fill(0)
        sampling[neg_idx] = len(sampling) / float(len(neg_idx))
       # self.WeightedSampling[pos_idx] = len(self.WeightedSampling) / float(len(pos_idx))
        sampling[pos_idx] = len(sampling) / float(len(pos_idx))
        self.WeightedSampling = sampling

        
        self.sum=np.insert(np.cumsum([len(self.audio[str(i)]) for i in self.sample]),0,0)
        print("data load fin")

        
    def __len__(self):
        return self.sum[-1]
    def __getitem__(self,index):
            vid=np.histogram(index,self.sum)#sum으로 누적으로 히스토그램이 깔려있음/ 그중에 index의 위치
            vid = np.where(vid[0]>0)[0][0]#몇번째 game을 쓸지!
            vframe=index-self.sum[vid]#그 게임 안에서의 몇번째 프레임인지
            game_id=str(self.sample[vid])

            window=[]#batch*7(window size)*3(highlight result)
            for idx in range(23): #7 : window size
                s_window=[]
                if vframe+idx<len(self.audio[game_id]):
                    s_window+=list((self.chat_result[game_id][vframe+idx]))#vframe의 chat
                    s_window+=list(self.audio[game_id][vframe+idx])#vframe의 image
                    s_window+=[self.video[game_id][vframe+idx]]
                else:
                    #s_window=[0,0,0]#padding value
                    s_window=[0]*1014
                window.append(s_window)


            label=int(self.real_result[game_id][vframe])
            return game_id,np.array(window),label

In [40]:
train=Mul_data('train')
val=Mul_data('val')

print(train[100])
sampler1 = torch.utils.data.sampler.WeightedRandomSampler(weights=train.WeightedSampling.tolist(), num_samples=44000)
train_loader=torch.utils.data.DataLoader(train,batch_size=32,sampler=sampler1)
# train_loader=torch.utils.data.DataLoader(train,batch_size=32)
val_loader=torch.utils.data.DataLoader(val,batch_size=128)

data load fin
data load fin
('102844412722519367', array([[ 0.        ,  0.        ,  0.        , ..., -2.99407219,
        -3.54347805,  0.01117067],
       [ 0.        ,  0.        ,  0.        , ..., -2.54867552,
        -3.03780565,  0.02793789],
       [ 0.        ,  0.        ,  0.        , ..., -0.24941073,
        -0.11264702,  0.03156882],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., -2.46387674,
        -2.8452097 ,  0.02793295],
       [ 0.        ,  0.        ,  0.        , ..., -3.52012466,
        -4.43745678,  0.03501002],
       [ 0.        ,  0.        ,  0.        , ..., -5.56696138,
        -3.35553581,  0.04068482]]), 0)


In [41]:
train[100][1].shape

(23, 1014)

In [42]:
input_size=1014
hidden_size=128
length=7
num_layers=3
class LSTM(nn.Module):
    def __init__(self):
        
        super().__init__()
        self._clf1 = nn.LSTM(input_size, hidden_size,3,batch_first=True)
        self._lin = nn.Sequential(nn.Linear(hidden_size, hidden_size),
                                 nn.Linear(hidden_size,2))

    def forward(self, x):
        x=x.cuda()
        hidden = Variable(torch.zeros(num_layers,x.size(0),hidden_size)).cuda() # (num_layers * num_directions, batch, hidden_size)
        cell = Variable(torch.zeros(num_layers,x.size(0),hidden_size)).cuda() # (num_layers * num_directions, batch, hidden_size)        out,hidden = self._clf1(x,h0)
        out,hidden = self._clf1(x,(hidden,cell))#batch*7*3
        feature = out[:,-1,:]
        out = self._lin(out[:,-1,:])
        return out,feature

In [43]:
model=LSTM().cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), 0.01,momentum=0.9,weight_decay=1e-4)


In [44]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [45]:
def fmeasure(output, target):
    _, pred = output.topk(1, 1, True, True)
    pred = pred.view(-1,1)
    target = target.view(-1,1)

    #overlap = ((pred== 1) + (target == 1)).gt(1)
    #overlap = overlap.view(-1,1)
    TP = len(np.where((pred==1)&(target==1)==True)[0]) # True positive
    FP = len(np.where((pred==1)&(target==0)==True)[0]) # Condition positive = TP + FN
    TN = len(np.where((pred==0)&(target==0)==True)[0])
    FN = len(np.where((pred==0)&(target==1)==True)[0])

    
    #overlap_len = overlap.data.long().sum()
    pred_len = pred.data.long().sum()
    gt_len   =  target.data.long().sum()

    return TP,FP,TN,FN,pred_len, gt_len,pred

In [48]:
weight_dir='./C_data+A_data+V_data/'


In [ ]:
if not os.path.exists(weight_dir):
    os.makedirs(weight_dir)
with open(weight_dir+'train_result','a') as f:
    f.write('=====result=======\n')
f1_best=0
for epoch in range(100):
    losses = AverageMeter()
    top1 = AverageMeter()
    print(epoch)
    model.train()
    for i, (g,inputs,labels) in enumerate(train_loader):
        inputs=inputs.float()
        inputs=inputs.cuda()
        labels=labels.cuda()
        optimizer.zero_grad()
        out,_=model(inputs)
        out=out.cuda()
        loss=criterion(out,labels)
        loss.backward()
        optimizer.step()
        
    model.eval()
    val_losses=AverageMeter()
    acc=0
    gt_sum=0
    tp_sum=0
    fp_sum=0
    fn_sum=0
    acc=0
    sum=0
    pred_sum=0
    with open(weight_dir+'train_result','a') as f:

        with torch.no_grad():
            for it, (g,inputs,labels) in enumerate(val_loader):
                inputs=inputs.float()
                inputs=inputs.cuda()
                labels=labels.cuda()
                out,_=model(inputs)
                out=out.cuda()
                loss=criterion(out,labels)
                val_losses.update(loss,labels.size(0))
                TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(out.cpu(),labels.cpu())
                tp_sum += TP
                fp_sum += FP
                fn_sum += FN
                pred_sum += pred_len
                gt_sum += gt_len
                acc=acc+TP+TN
                sum+=len(out)
            if tp_sum>0 and fp_sum>0 and fn_sum>0:
                precision = tp_sum/(tp_sum+fp_sum)
                recall = tp_sum / (tp_sum+fn_sum)
                f1 = (2*precision*recall / (precision + recall))
                accuracy=acc/sum
                print("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                f.write("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))
                if f1_best<f1:
                    f.write("== best epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                    torch.save(model.state_dict(),'{}'.format(weight_dir+"best"))
                    f1_best=f1

            else:
                print("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,0,0,0))
                f.write("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,0,0,0))
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))                
            







0
epoch 0 train_loss : 0 , val_loss : 0.6452922821044922,p 0.2901206973625391, r 0.7163355408388521, f 0.4129812281259943

1
epoch 1 train_loss : 0 , val_loss : 0.4511004090309143,p 0.5520361990950227, r 0.4578366445916115, f 0.500543019186678

2
epoch 2 train_loss : 0 , val_loss : 0.5397719144821167,p 0.4202429149797571, r 0.6874172185430464, f 0.5216080402010049

3
epoch 3 train_loss : 0 , val_loss : 0.4119870066642761,p 0.5821478382147838, r 0.4607064017660044, f 0.5143561306223043

4
epoch 4 train_loss : 0 , val_loss : 0.7676252722740173,p 0.26316133673402653, r 0.8883002207505519, f 0.4060340043388326

5
epoch 5 train_loss : 0 , val_loss : 0.40578827261924744,p 0.5741367637102234, r 0.37439293598233997, f 0.45323356493853556

6
epoch 6 train_loss : 0 , val_loss : 0.394549161195755,p 0.5824446267432322, r 0.47019867549668876, f 0.5203371198241115

7
epoch 7 train_loss : 0 , val_loss : 0.506081759929657,p 0.41790625877069887, r 0.6573951434878588, f 0.5109814687714482

8
epoch 8 tra

KeyboardInterrupt: 

In [49]:
test=Mul_data('test')
test_loader=torch.utils.data.DataLoader(test,batch_size=32)
dataset=weight_dir+'best'
checkpoint=torch.load(dataset,map_location='cuda:0')
model.load_state_dict(checkpoint)
model.eval()
pred_sum = 0#model output
gt_sum = 0#label
tp_sum=0
fp_sum=0
fn_sum=0
acc=0
sum=0
result={}
with torch.no_grad():
    for it, (game_id,inputs,labels) in enumerate(test_loader):
        inputs=inputs.float()
        labels=labels
        output,_=model(inputs)
        TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(output.cpu(),labels.cpu())
        for idx,g in enumerate(game_id):
            if g not in result.keys():
                result[g]=pred[idx].tolist()
            else:
                result[g]+=pred[idx].tolist()
        print(TP,FP,TN,FN,pred_len, gt_len)
        tp_sum += TP
        fp_sum += FP
        fn_sum += FN
        pred_sum += pred_len
        gt_sum += gt_len
        acc=acc+TP+TN
        sum+=len(output)
    with open(weight_dir+'/train_result','a') as f:
        if tp_sum>0 and fp_sum>0 and fn_sum>0:
            precision = tp_sum/(tp_sum+fp_sum)
            recall = tp_sum / (tp_sum+fn_sum)
            f1 = (2*precision*recall / (precision + recall)) * 100
            accuracy=acc/sum
            print( tp_sum, fp_sum, fn_sum)
            print('[{}/{}], prec:{}, recall:{}, f1:{}, acc: {}'.format(it, len(test_loader), precision, recall, f1,accuracy))
            f.write('{}, prec:{}, recall:{}, f1:{}, acc : {}\n'.format(dataset, precision, recall, f1,accuracy))

data load fin
0 2 30 0 tensor(2) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
5 0 27 0 tensor(5) tensor(5)
3 0 23 6 tensor(3) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 12 20 0 tensor(12) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 23 9 tensor(0) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 16 16 tensor(0) tensor(16)
7 2 13 10 tensor(9) tensor(17)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
10 10 10 2 tensor(20) tensor(12)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
6 0 25 1 tensor(6) tensor(7)
11 0 14 7 tensor(11) tensor(18)
0 0 24 8 tensor(0) tensor(8)
0 0 32 0 tensor(0) tensor(0)
0 0 19 13 tensor

0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 1 31 0 tensor(1) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 22 10 tensor(0) tensor(10)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
12 0 15 5 tensor(12) tensor(17)
13 3 16 0 tensor(16) tensor(13)
0 14 8 10 tensor(14) tensor(10)
0 13 19 0 tensor(13) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
6 0 18 8 tensor(6) tensor(14)
2 0 21 9 tensor(2) tensor(11)
0 0 22 10 tensor(0) tensor(10)
0 0 30 2 tensor(0) tensor(2)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 1 31 0 tensor(1) tensor(0)
15 4 13 0 tensor(19) tensor(15)
7 0 23 2 tensor(7) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
2 0 29 1 tensor(2) tensor(3)
21 0 0 11 tensor(21) tensor(32)
10 0 12 10 tensor(10) tensor(20)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(

0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 4 28 0 tensor(4) tensor(0)
0 24 1 7 tensor(24) tensor(7)
1 4 0 27 tensor(5) tensor(28)
0 4 19 9 tensor(4) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 2 30 0 tensor(2) tensor(0)
0 0 32 0 tensor(0) tensor(0)
5 11 10 6 tensor(16) tensor(11)
0 2 30 0 tensor(2) tensor(0)
0 9 23 0 tensor(9) tensor(0)
0 0 23 9 tensor(0) tensor(9)
0 1 22 9 tensor(1) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 7 21 4 tensor(7) tensor(4)
0 6 26 0 tensor(6) tensor(0)
10 8 14 0 tensor(18) tensor(10)
12 9 11 0 tensor(21) tensor(12)
0 13 19 0 tensor(13) tensor(0)
0 8 24 0 tensor(8) tensor(0)
0 19 13 0 tensor(19) tensor(0)
0 25 7 0 tensor(25) tensor(0)
0 2 27 3 tensor(2) tensor(3)
0 0 9 23 tensor(0) tensor(23)
3 11 12 6 tensor(14) tensor(9)
0 11 21 0 tensor(11) tensor(0)
0 6 26 0 tensor(6) tensor(0)
2 2 23 5 tensor(4) tensor(7)
0 0 26 6 tensor(0) tensor(6)
0 0 20 12 tensor(0) tensor(12)
0 0 12 20 tensor(0) 

4 0 27 1 tensor(4) tensor(5)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 1 31 0 tensor(1) tensor(0)
0 0 28 4 tensor(0) tensor(4)
0 2 26 4 tensor(2) tensor(4)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 5 27 0 tensor(5) tensor(0)
6 0 4 22 tensor(6) tensor(28)
0 0 27 5 tensor(0) tensor(5)
0 0 32 0 tensor(0) tensor(0)
13 0 9 10 tensor(13) tensor(23)
11 0 18 3 tensor(11) tensor(14)
0 1 25 6 tensor(1) tensor(6)
7 1 24 0 tensor(8) tensor(7)
29 2 1 0 tensor(31) tensor(29)
0 1 31 0 tensor(1) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
22 1 9 0 tensor(23) tensor(22)
0 0 25 7 tensor(0) tensor(7)
0 0 32 0 tensor(0) tensor(0)
0 0 26 6 tensor(0) tensor(6)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
6 0 20 6 tensor(6) tensor(12)
21 5 6 0 tensor(26) tensor(21)
0 0 24 8 tensor(0) tensor(8)


In [50]:
def fmeasure2(frames,label):
    average = [0,0,0,0,0]
    for key in frames.keys():
        TP = len(np.where((np.array(frames[key])==1)&(label[key]==1)==True)[0])
        FP = len(np.where((np.array(frames[key])==1)&(label[key]==0)==True)[0])
        TN = len(np.where((np.array(frames[key])==0)&(label[key]==0)==True)[0])
        FN = len(np.where((np.array(frames[key])==0)&(label[key]==1)==True)[0])
        precision = TP/(TP+FP)
        recall = TP/(TP+FN)
        accuracy = (TP+TN)/(TP+FN+FP+TN)
        if precision==0 and recall == 0:
            print('!')
        else:
            f1 = (2*precision*recall / (precision + recall))
            print(key)
            print('precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(precision,recall,f1,accuracy))
            average[0]+= precision
            average[1]+= recall
            average[2]+= f1
            average[3]+= accuracy
            average[4]+=1
    print('==precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(average[0]/average[4],average[1]/average[4],average[2]/average[4],average[3]/average[4]))
with open('./label/label.pickle',"rb") as f4:  
    real_result=pickle.load(f4)
fmeasure2(result,real_result)

102844212431058132
precision : 0.6606334841628959, recall : 0.38320209973753283, f1 : 0.48504983388704326, accuracy : 0.8741883116883117
102844341902586509
precision : 0.4424131627056673, recall : 0.9565217391304348, f1 : 0.605, accuracy : 0.852198316183349
102844401152267937
precision : 0.19593613933236576, recall : 0.3358208955223881, f1 : 0.24747937671860684, accuracy : 0.6413280908693753
102844212430927059
precision : 0.6524300441826215, recall : 0.6712121212121213, f1 : 0.6616878267363705, accuracy : 0.88088351301604
102844412708953395
precision : 0.5477855477855478, recall : 0.8514492753623188, f1 : 0.6666666666666666, accuracy : 0.8857003891050583
102844212429944013
precision : 0.6589861751152074, recall : 0.38544474393531, f1 : 0.4863945578231292, accuracy : 0.852394916911046
102844341912679064
precision : 0.46715328467153283, recall : 0.8495575221238938, f1 : 0.6028257456828886, accuracy : 0.8695876288659794
102844235753749959
precision : 0.25464684014869887, recall : 0.348600

In [10]:
a=torch.transpose(b,1,2)

In [11]:
a

tensor([[[0.1380, 0.0111],
         [0.8294, 0.4059],
         [0.0521, 0.1911]]])

In [11]:
x=[]

In [9]:
with open('../data/chat_feature_pred_128_train.json',"rb") as f1:  
    chat_result=json.load(f1)

In [24]:
len(chat_result['102844235753356742'])

1654

In [21]:
        with open('../data/audio_energy_2_normaized.pickle',"rb") as f3:  
            audio_result=pickle.load(f3)

In [25]:
audio_result['102844235753356742'][1]

4.277879204882054e-07

In [8]:
        with open('../data/audio_H.pickle',"rb") as f2:  
            image_result=pickle.load(f2)

In [10]:
len(image_result['102844235753356742'])

16571